## 导入库和数据

In [ ]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
on_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_online_stage1_train\ccf_online_stage1_train.csv')
off_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_train\ccf_offline_stage1_train.csv')
# oftid_train=pd.read_csv(r'D:\Data\TCForNewComer\off_train_sameUser_id_test.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')
samplt=pd.read_csv(r'D:\Data\TCForNewComer\sample_submission.csv')

In [ ]:
off_train=off_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)
off_train.head()

In [ ]:
test['Date_received']=test['Date_received'].astype(np.int64)
test['Date_received']=test['Date_received'].apply(lambda x: str(x))
test.index=np.arange(0,len(test),1)
test.head()

In [ ]:
on_train=on_train.sort_values(by=['User_id'])
on_train.index=np.arange(0,len(on_train),1)
on_train.head()

In [ ]:
col='Merchant_id'
a=set(on_train[col].values)
b=set(test[col].values)
c=set(oftid_train[col].values)
print(len(b))
print(len(a))
print(len(c))

print(len(a&b))
print(len(b&c))
print(len(c-b))

## 增加一列之前没用优惠券是否购买过相同商品
- 可能是日常用品，或者零食
- 可能是耐用品，然后买了之后就不会再买

In [ ]:
oftid_train=off_train.copy()
indexs=oftid_train[(oftid_train['Coupon_id']=='null')&(oftid_train['Date']!='null')].index
odinary=oftid_train.iloc[indexs]

In [ ]:
oftid_train.drop(indexs,axis=0,inplace=True)

In [ ]:
oftid_train['ord_buy']=0#增加一列

In [ ]:
odinary.index=np.arange(0,len(odinary),1)
oftid_train.index=np.arange(0,len(oftid_train),1)

In [ ]:
#计算同一件商品半年内普通购买次数
from ipykernel import kernelapp as app
i=0
j=10
length=len(oftid_train)
for index in oftid_train.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=oftid_train.iloc[index]['User_id']
    mid=oftid_train.iloc[index]['Merchant_id']
    if len(odinary[odinary['User_id']==uid].index)!=0:
        for ind in odinary[odinary['User_id']==uid].index:
            if odinary.iloc[ind]['Merchant_id']==mid:
                oftid_train.loc[index,'ord_buy']+=1
    i+=1

In [ ]:
oftid_train.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\oftid_addordinarybuy_train.csv',index=False)

In [ ]:
test['ord_buy']=0#增加一列
test.index=np.arange(0,len(test),1)

#计算同一件商品半年内普通购买次数
i=0
j=10
length=len(test)
for index in test.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=test.iloc[index]['User_id']
    mid=test.iloc[index]['Merchant_id']
    if len(odinary[odinary['User_id']==uid].index)!=0:
        for ind in odinary[odinary['User_id']==uid].index:
            if odinary.iloc[ind]['Merchant_id']==mid:
                test.loc[index,'ord_buy']+=1
    i+=1

In [ ]:
test.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\test_addordinarybuy.csv',index=False)

### 线下领取优惠券张数和不同优惠券的张数和使用数目

In [1]:
import pandas as pd
import numpy as np
or_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\oftid_addordinarybuy_train.csv')
or_test=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\test_addordinarybuy.csv')
off_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_train\ccf_offline_stage1_train.csv')
or_train=or_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)
or_train.index=np.arange(0,len(or_train),1)
or_test.index=np.arange(0,len(or_test),1)

In [2]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return int(x.split(':')[0])-int(x.split(':')[1])
    #将满减转化为小数
or_train['Discount_rate']=or_train['Discount_rate'].apply(Discount_rate)

In [3]:
or_train['Coupon_num']=0                   #领取张数
or_train['Coupon_nouse_num']=0             #领取没有使用张数
or_train['Coupon_use_num']=0               #使用张数
or_train['Coupon_use_prob']=0              #使用概率
i=0
j=10
length=len(set(or_train['User_id'].values))
for id in set(or_train['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    mid_Ser=or_train[or_train['User_id']==id]['Date']
    lengths=len(mid_Ser[mid_Ser=='null'])
    or_train.loc[mid_Ser.index,'Coupon_num']=len(mid_Ser)
    or_train.loc[mid_Ser.index,'Coupon_nouse_num']=lengths
    or_train.loc[mid_Ser.index,'Coupon_use_num']=len(mid_Ser)-lengths
    or_train.loc[mid_Ser.index,'Coupon_use_prob']=float(len(mid_Ser)-lengths)/len(mid_Ser)
    i+=1
or_train.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\or_train.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

In [4]:
or_test['Coupon_num']=0                   #领取张数
or_test['Coupon_nouse_num']=0             #领取没有使用张数
or_test['Coupon_use_num']=0               #使用张数
or_test['Coupon_use_prob']=0              #使用概率
i=0
j=10
length=len(set(or_test['User_id'].values))
for id in set(or_test['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    mid_Ser=or_train[or_train['User_id']==id]['Date']
    if len(mid_Ser)!=0:
        lengths=len(mid_Ser[mid_Ser=='null'])
        test_Ser=or_test[or_test['User_id']==id]
        or_test.loc[test_Ser.index,'Coupon_num']=len(mid_Ser)
        or_test.loc[test_Ser.index,'Coupon_nouse_num']=lengths
        or_test.loc[test_Ser.index,'Coupon_use_num']=len(mid_Ser)-lengths
        or_test.loc[test_Ser.index,'Coupon_use_prob']=float(len(mid_Ser)-lengths)/len(mid_Ser)
    i+=1
or_test.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\or_test.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

### 使用优惠券面额大于100的次数和小于100的次数

In [3]:
import pandas as pd
import numpy as np
oc_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\train.csv')
oc_test=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\test.csv')
oc_train=oc_train.sort_values(by=['User_id'])
oc_train.index=np.arange(0,len(oc_train),1)
oc_test.index=np.arange(0,len(oc_test),1)

In [24]:
oc_train['Coupon_greaterthan100']=0                   #使用面额大于100的张数
oc_train['Coupon_lessthan100']=0                      #使用面额小于100的张数
oc_train['Coupon_lep100']=0                          #使用面额大于100的张数占使用张数的比重
oc_train['Coupon_grp100']=0                          #使用面额小于100的张数占使用张数的比重
oc_train['Avg_saler_Coupon']=0                       #核销每个商家多少张优惠券
oc_train['Avg_distance']=0                          #用户核销优惠券的平均距离
oc_train['shop_count']=0                            #核销商家数量
oc_train['shop_pro']=0                              #商家优惠券占总的比例
oc_train['Coupon_diffcount']=0                      #不同优惠券的数量
oc_train['Coupon_diffpro']=0                        #不同优惠券的占比
oc_train['same_Coupon_count']=0                      #相同优惠券领取的次数
i=0
j=10
length=len(set(oc_train[oc_train['Date']==1]['User_id'].values))
for id in set(oc_train[oc_train['Date']==1]['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    discount_rate=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]['Discount_rate']
    indexs=oc_train[oc_train['User_id']==id].index
    gr100=len(discount_rate[discount_rate>100])
    le100=len(discount_rate[discount_rate<100])
    use_count=oc_train.loc[discount_rate.index,'Coupon_use_num'].max()
    oc_train.loc[indexs,'Coupon_greaterthan100']=gr100
    oc_train.loc[indexs,'Coupon_lessthan100']=le100
    oc_train.loc[indexs,'Coupon_grp100']=gr100/use_count
    oc_train.loc[indexs,'Coupon_lep100']=le100/use_count
    
    customer=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]
    oc_train.loc[indexs,'shop_count']=len(set(customer['Distance'].values))
    oc_train.loc[indexs,'Coupon_diffcount']=len(set(customer['Discount_rate'].values))
    for dis in set(customer['Distance'].values):                     #不同的商家
        cus=customer[customer['Distance']==dis]
        indexss=oc_train[(oc_train['User_id']==id)&(oc_train['Distance']==dis)].index
        oc_train.loc[indexss,'Avg_saler_Coupon']=len(cus)
        oc_train.loc[indexss,'shop_pro']=len(cus)/len(customer)
        for rat in set(customer['Discount_rate'].values):             #不同优惠额度
            Rate=customer[customer['Discount_rate']==rat]
            indexsss=oc_train[(oc_train['User_id']==id)&(oc_train['Distance']==dis)&(oc_train['Discount_rate']==rat)].index
            oc_train.loc[indexsss,'Coupon_diffpro']=float(len(Rate))/len(cus)
    
    for coupid in set(oc_train[oc_train['User_id']==id]['Coupon_id'].values):
        indexx=oc_train[(oc_train['User_id']==id)&(oc_train['Coupon_id']==coupid)].index
        oc_train.loc[indexx,'same_Coupon_count']=len(indexx)
        
    oc_train.loc[indexs,'Avg_distance']=customer['Distance'].mean()
    i+=1
oc_train.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\oc_train.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

In [5]:
oc_test['Coupon_greaterthan100']=0                   #使用面额大于100的张数
oc_test['Coupon_lessthan100']=0                      #使用面额小于100的张数
oc_test['Coupon_lep100']=0                          #使用面额大于100的张数占使用张数的比重
oc_test['Coupon_grp100']=0                          #使用面额小于100的张数占使用张数的比重
oc_test['Avg_saler_Coupon']=0                       #核销每个商家多少张优惠券
oc_test['Avg_distance']=0                          #用户核销优惠券的平均距离
oc_test['shop_count']=0                            #核销商家数量
oc_test['shop_pro']=0                              #商家优惠券占总的比例
oc_test['Coupon_diffcount']=0                      #不同优惠券的数量
oc_test['Coupon_diffpro']=0                        #不同优惠券的占比
oc_test['same_Coupon_count']=0                      #相同优惠券领取的次数
i=0
j=10
length=len(set(oc_test['User_id'].values))
for id in set(oc_test['User_id'].values):
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%',end='-')
        j=j+10
    
    discount_rate=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]['Discount_rate']
    indexs=oc_test[oc_test['User_id']==id].index
    gr100=len(discount_rate[discount_rate>100])
    le100=len(discount_rate[discount_rate<100])
    use_count=oc_train.loc[discount_rate.index,'Coupon_use_num'].max()
    oc_test.loc[indexs,'Coupon_greaterthan100']=gr100
    oc_test.loc[indexs,'Coupon_lessthan100']=le100
    if len(discount_rate)!=0:
        oc_test.loc[indexs,'Coupon_grp100']=gr100/use_count
        oc_test.loc[indexs,'Coupon_lep100']=le100/use_count
    
    customer=oc_train[(oc_train['User_id']==id)&(oc_train['Date']==1)]
    oc_test.loc[indexs,'shop_count']=len(set(customer['Distance'].values))
    oc_test.loc[indexs,'Coupon_diffcount']=len(set(customer['Discount_rate'].values))
    for dis in set(oc_test[(oc_test['User_id']==id)]['Distance'].values):                     #不同的商家
        cus=customer[customer['Distance']==dis]
        indexss=oc_test[(oc_test['User_id']==id)&(oc_test['Distance']==dis)].index
        oc_test.loc[indexss,'Avg_saler_Coupon']=len(cus)
        if len(customer)!=0:
            oc_test.loc[indexss,'shop_pro']=float(len(cus))/len(customer)
        
        for rat in set(oc_test[(oc_test['User_id']==id)]['Discount_rate'].values):             #不同优惠额度
            Rate=customer[customer['Discount_rate']==rat]
            indexsss=oc_test[(oc_test['User_id']==id)&(oc_test['Distance']==dis)&(oc_test['Discount_rate']==rat)].index
            if len(cus)!=0:
                oc_test.loc[indexsss,'Coupon_diffpro']=float(len(Rate))/len(cus)
    
    for coupid in set(oc_test[oc_test['User_id']==id]['Coupon_id'].values):
        indexx=oc_test[(oc_test['User_id']==id)&(oc_test['Coupon_id']==coupid)].index
        oc_test.loc[indexx,'same_Coupon_count']=len(indexx)
    if len(customer['Distance'])!=0:
        oc_test.loc[indexs,'Avg_distance']=customer['Distance'].mean()
    i+=1
oc_test.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\oc_test.csv',index=False)

. 10 %-.. 20 %-... 30 %-.... 40 %-..... 50 %-...... 60 %-....... 70 %-........ 80 %-......... 90 %-

##  商家提供的优惠券种类和张数

### 算有几家店铺，店铺中的商品种类，店铺的优惠券种类和张数，顾客经常使用优惠券的商店，对店铺进行编码，优惠券领取的张数，优惠券使用的张数和占比，发放优惠券的平均面额

In [ ]:
import pandas as pd
import numpy as np
os_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\oc_train.csv')
os_test=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\oc_test.csv')

In [ ]:
def shopn(id):                        #计算店铺数量
    for mid in  id:
        indexs=[]
        indexs.extend(list(os_train[os_train['Merchant_id']==mid].index))
        avg=os_train[os_train['Merchant_id']==mid]['Discount_rate'].mean()

        for uid in set(os_train[os_train['Merchant_id']==mid]['User_id'].values):
            dis=os_train[(os_train['Merchant_id']==mid)&(os_train['User_id']==uid)]['Distance'].max()
            merchantid=set()
            for ind in os_train[(os_train['User_id']==uid)&(os_train['Distance']==int(dis))].index:
                if (abs(os_train.loc[ind,'Discount_rate']-avg)<100):
                    indexs.extend([ind])
                    if ind not in indexs:
                        merchantid.add(os_train.loc[ind,'Merchant_id'])
            if len(merchantid)!=0:
                indexs.extend(shopn(merchantid)) 
    return indexs

## 增加用券在线购买和普通购买、领取优惠券、点击的列

In [ ]:
test=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\test_addordinarybuy.csv')

In [ ]:
oftid_train['online_buy']=0#增加一列
oftid_train['online_col']=0
oftid_train['online_click']=0
test['online_buy']=0#增加一列
test['online_col']=0
test['online_click']=0
# odinary.index=np.arange(0,len(odinary),1)
# ontid_train.index=np.arange(0,len(ontid_train),1)
oftid_train.index=np.arange(0,len(oftid_train),1)
test.index=np.arange(0,len(test),1)

In [ ]:
i=0
j=10
length=len(oftid_train)
for index in oftid_train.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=oftid_train.iloc[index]['User_id']
    if len(on_train[on_train['User_id']==uid].index)!=0:
        for ind in on_train[on_train['User_id']==uid].index:
            if on_train.iloc[ind]['Action']==1:
                oftid_train.loc[index,'online_buy']+=1
            elif on_train.iloc[ind]['Action']==2:
                oftid_train.loc[index,'online_col']+=1
            else:
                oftid_train.loc[index,'online_click']+=1
    i+=1

In [ ]:
oftid_train.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\ofon_coupod_buy_train.csv',index=False)

In [ ]:
i=0
j=10
length=len(test)
for index in test.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,120.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=test.iloc[index]['User_id']
    if len(on_train[on_train['User_id']==uid].index)!=0:
        for ind in on_train[on_train['User_id']==uid].index:
            if on_train.iloc[ind]['Action']==1:
                test.loc[index,'online_buy']+=1
            elif on_train.iloc[ind]['Action']==2:
                test.loc[index,'online_col']+=1
            else:
                test.loc[index,'online_click']+=1
    i+=1

In [ ]:
test.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\test_ofon_coupod_buy.csv',index=False)

In [ ]:
off_train=oftid_train.copy()

***

# 导入增加几列以后的数据集

In [5]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
off_train=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\or_train.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\or_test.csv')

In [7]:
off_train=off_train.sort_values(by=['User_id'])
off_train['Date_received']=off_train['Date_received'].apply(lambda x: str(x))
off_train.index=np.arange(0,len(off_train),1)
off_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,ord_buy,Coupon_num,Coupon_nouse_num,Coupon_use_num,Coupon_use_prob
0,4,1433,8735,25.00,10,20160214,null,0,2,2,0,0.0
1,4,1469,2902,0.95,10,20160607,null,0,2,2,0,0.0
2,35,3381,1807,270.00,0,20160130,null,0,4,4,0,0.0
3,35,3381,9776,5.00,0,20160129,null,0,4,4,0,0.0
4,35,3381,11951,180.00,0,20160129,null,0,4,4,0,0.0


In [6]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return int(x.split(':')[0])-int(x.split(':')[1])
#将满减转化为小数
test['Discount_rate']=test['Discount_rate'].apply(Discount_rate)
test['Date_received']=test['Date_received'].astype(np.int64)
test['Date_received']=test['Date_received'].apply(lambda x: str(x))
test.index=np.arange(0,len(test),1)
test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,ord_buy,Coupon_num,Coupon_nouse_num,Coupon_use_num,Coupon_use_prob
0,4129537,450,9983,25.0,1,20160712,0,2,2,0,0.0
1,6949378,1300,3429,25.0,null,20160706,0,1,0,1,1.0
2,2166529,7113,6928,180.0,5,20160727,0,1,1,0,0.0
3,2166529,7113,1808,90.0,5,20160727,0,1,1,0,0.0
4,6172162,7605,6500,29.0,2,20160708,0,1,1,0,0.0


## 选取间隔日期大于15天，标记为负样本

In [8]:
p_train=off_train[off_train['Date']!='null'].copy()#正样本
n_train=off_train[off_train['Date']=='null'].copy()#负样本
p_train.index=np.arange(0,len(p_train),1)
n_train.index=np.arange(0,len(n_train),1)
#转换为时间格式
p_train['Date_received_copy']=p_train['Date_received'].copy()

p_train['Date_received_copy']=pd.to_datetime(p_train['Date_received_copy'])
p_train['Date']=pd.to_datetime(p_train['Date'])
# n_train['Date_received']=pd.to_datetime(n_train['Date_received'])
# test['Date_received']=pd.to_datetime(test['Date_received'])

p_train['date']=(p_train['Date']-p_train['Date_received_copy']).astype('timedelta64[D]')
p_train.loc[p_train[p_train['date']<=15].index,'date']=1
p_train.loc[p_train[p_train['date']>15].index,'date']=0
p_train['Date']=p_train['date'].copy()
p_train.drop(['date','Date_received_copy'],axis=1,inplace=True)
p_train['Date']=p_train['Date'].astype(np.int64)

In [9]:
#转换为星期几x.weekday()+1
test['Date_received']=(test['Date_received'].apply(lambda x: int(x[5:8]))).copy()
#正样本为1，负样本为0
n_train['Date']=0
train=pd.concat([p_train,n_train],axis=0)
train['Date_received']=(train['Date_received'].apply(lambda x: int(x[5:8]))).copy()
train['Date_received']=train['Date_received'].astype(np.int64)
train=train.sort_values(by=['User_id'])
train.index=np.arange(0,len(train),1)

## 转化满减为小数折扣

In [ ]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return int(x.split(':')[0])-int(x.split(':')[1])

In [12]:
train=train.replace('null',np.nan)
test=test.replace('null',np.nan)

In [ ]:
#将满减转化为小数
train['Discount_rate']=train['Discount_rate'].apply(Discount_rate)

In [13]:
train=train.fillna(method='ffill')

In [14]:
train['Distance']=train['Distance'].astype(np.int64)
train['Coupon_id']=train['Coupon_id'].astype(np.int64)

In [15]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1053282 entries, 0 to 1053281
Data columns (total 12 columns):
User_id             1053282 non-null int64
Merchant_id         1053282 non-null int64
Coupon_id           1053282 non-null int64
Discount_rate       1053282 non-null float64
Distance            1053282 non-null int64
Date_received       1053282 non-null int64
Date                1053282 non-null int64
ord_buy             1053282 non-null int64
Coupon_num          1053282 non-null int64
Coupon_nouse_num    1053282 non-null int64
Coupon_use_num      1053282 non-null int64
Coupon_use_prob     1053282 non-null float64
dtypes: float64(2), int64(10)
memory usage: 104.5 MB


## 不同类型数据的影响

In [ ]:
train['Merchant_id']=train['Merchant_id'].astype(np.object)
train['User_id']=train['User_id'].astype(np.object)
train['Date_received']=train['Date_received'].astype('category')
test['Merchant_id']=test['Merchant_id'].astype(np.object)
test['Coupon_id']=test['Coupon_id'].astype(np.object)
test['Distance']=test['Distance'].astype(np.object)
test['Date_received']=test['Date_received'].astype('category')

In [ ]:
test['User_id']=test['User_id'].astype(np.object)

### 填充测试集的空缺值

In [16]:
test=test.fillna(method='ffill')

In [ ]:
test['Discount_rate']=test['Discount_rate'].apply(Discount_rate)

In [17]:
test['Merchant_id']=test['Merchant_id'].astype(np.int64)
test['Coupon_id']=test['Coupon_id'].astype(np.int64)
test['Distance']=test['Distance'].astype(np.int64)

In [18]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 113640 entries, 0 to 113639
Data columns (total 11 columns):
User_id             113640 non-null int64
Merchant_id         113640 non-null int64
Coupon_id           113640 non-null int64
Discount_rate       113640 non-null float64
Distance            113640 non-null int64
Date_received       113640 non-null int64
ord_buy             113640 non-null int64
Coupon_num          113640 non-null int64
Coupon_nouse_num    113640 non-null int64
Coupon_use_num      113640 non-null int64
Coupon_use_prob     113640 non-null float64
dtypes: float64(2), int64(9)
memory usage: 10.4 MB


In [19]:
train.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\train.csv',index=False)
test.to_csv(r'D:\Data\TCForNewComer\deal\ALLuserid\test.csv',index=False)

In [ ]:
#行最大最小归一化
XX=train.copy()
for col in ['Merchant_id','Coupon_id']:
    max=XX[col].max()
    min=XX[col].min()
    XX[col]=XX[col].apply(lambda x: ((x-min)/(max-min)))

In [ ]:
XX.head()

In [ ]:
cor=XX.corr()
cor[cor<0.01]=0
cor

## 距离统计分布

In [ ]:
import seaborn as sns 
%matplotlib inline

sns.countplot(x='Distance',hue='Date',data=train)

In [ ]:
import seaborn as sns
%matplotlib inline

sns.countplot(x='Date_received',hue='Date',data=train)

In [ ]:
import seaborn as sns 
%matplotlib inline
plt.figure(figsize=(10,6))
sns.countplot(x='Discount_rate',hue='Date',data=train)
plt.xticks(rotation=90)
plt.show()

In [ ]:
train.head()

In [ ]:
a=set(train['Coupon_id'].values)
c=set(test['Coupon_id'].values)
len(c)

In [ ]:
corr=train.corr()
corr

In [ ]:
train

In [ ]:
combine=[train,test]
for data in combine:
    data.drop(['Date_received'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
a=set(train['Merchant_id'].values)
c=set(test['Merchant_id'].values)
len(c)

In [ ]:
len(a&c)

# 训练模型

In [ ]:
K=3
from sklearn.metrics import roc_auc_score,auc
from time import sleep
import winsound
from sklearn.model_selection import StratifiedKFold
# kf = KFold(n_splits = K,random_state = 1,shuffle = True)
kf=StratifiedKFold(n_splits = K,random_state = 90,shuffle = True)
X=train.drop(['Date'],axis=1)
y=train['Date']

In [ ]:
X=X.drop(['online_click'],axis=1)

## 归一化

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
x_predict=test
# 1. min max scaler
min_max_scaler = MinMaxScaler()
x_predict_min_max_scaled = min_max_scaler.fit_transform(x_predict)

# 2. standard scaler
std_scaler = StandardScaler()
x_predict_std_scaled = std_scaler.fit_transform(x_predict)

In [ ]:
# 1. min max scaler
min_max_scaler = MinMaxScaler()
# x_train_min_max_scaled = min_max_scaler.fit_transform(x_train)
# x_test_min_max_scaled = min_max_scaler.transform(x_test)
X_test_min_max_scaled=min_max_scaler.fit_transform(X)#整个数据集
# 2. standard scaler
std_scaler = StandardScaler()
# x_train_std_scaled = std_scaler.fit_transform(x_train)
# x_test_std_scaled = std_scaler.transform(x_test)
X_test_std_scaled=std_scaler.fit_transform(X)

In [ ]:
import xgboost as xgb
#全部  0.5552 learning_rate=0.09
model=xgb.XGBClassifier(n_estimators=200,
                        max_depth=2,#6
                        objective="binary:logistic",
                        learning_rate=0.09, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.4,
                        scale_pos_weight=1.6,
                        gamma=9,
                        seed=100,
                        reg_alpha=8,
                        reg_lambda=1.3)

In [ ]:
def Avg_auc(pred):
    aucc=0
    i=0
    df=train.copy()
    df['y_valid_pred']=pred
    for j in list(set(df['Coupon_id'].values)):
        df_1=df[df['Coupon_id']==j]
        if len(np.unique(df_1['Date']))==1:
            continue
        aucc=aucc+roc_auc_score(df_1['Date'],df_1['y_valid_pred'])
        i+=1
    aucc=aucc/i
    df.drop(['y_valid_pred'],axis=1,inplace=True)
    return aucc

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=5)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

In [ ]:
from sklearn.ensemble import AdaBoostClassifier#星期0.626
model=AdaBoostClassifier(n_estimators=400,learning_rate=1,random_state=100)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
pred_test=model.predict_proba(test)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

In [ ]:
y_test_pred=0
y_valid_pred=0*y
fit_model=[]
# XMIN=pd.DataFrame(X_test_std_scaled,columns=['Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received'])
for i,(train_index,test_index) in enumerate(kf.split(X,y)):
    y_train,y_valid=y.iloc[train_index],y.iloc[test_index]
    x_train,x_valid=X.iloc[train_index],X.iloc[test_index]
    print('\nFlod:',i,end=':')
    fitted_model=model.fit(x_train,y_train)
    pred= fitted_model.predict_proba(x_valid)[:,1]
    y_valid_pred.iloc[test_index]=pred
    print(pd.DataFrame(pred).head(1))
    
    y_test_pred+=fitted_model.predict_proba(test)[:,1]
    fit_model.append(fitted_model)
    winsound.Beep(600,1000)
    sleep(1)
y_test_pred/=K
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))

In [ ]:
tt=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')

In [ ]:
sub=pd.DataFrame()
sub['User_id']=tt['User_id']
sub['Coupon_id']=tt['Coupon_id']
sub['Date_received']=tt['Date_received']

In [ ]:
sub['Probability']=y_test_pred

In [ ]:
sub.to_csv(r'D:\Data\TCForNewComer\ALLuseridS1pred_result.csv',float_format='%.1f',index=False)

In [ ]:
y_train['a']=np.random.normal(len(y_train))

In [ ]:
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 1, 0, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
y_true.astype(np.int64)

In [ ]:
train.head()